In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf 

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
x = dataset.iloc[:,3:-1].values 
y = dataset.iloc[:,-1].values

In [3]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [4]:
print(y)

[1 0 1 ... 1 1 0]


Encoding Categorical data

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:,2])
x

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
x = np.array(ct.fit_transform(x))
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


Split the train and test dataset

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=1)

Feature Scaling

In [20]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test  = sc.transform(x_test)

Building the ANN

In [21]:
ann = tf.keras.models.Sequential()

Adding the input layer and first hidden layer 

In [22]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Second hidden layer 

In [23]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Output layer

In [24]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Training the ANN

In [25]:
ann.compile(optimizer='adam' , loss='binary_crossentropy', metrics= ['accuracy'])

In [26]:
ann.fit(x_train,y_train, batch_size= 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5377 - accuracy: 0.7943
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4756 - accuracy: 0.7972
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4559 - accuracy: 0.7972
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4446 - accuracy: 0.7972
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4381 - accuracy: 0.7972
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4337 - accuracy: 0.7972
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4310 - accuracy: 0.7972
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4290 - accuracy: 0.7972
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4274 - accuracy: 0.8101
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4260 - accura

predicting the result for single observation

In [27]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 67ms/step
[[False]]


Predicting the test results

In [31]:
y_pred = ann.predict(sc.fit_transform(x_test))
y_pred = (y_pred > 0.5)
print(np.concatenate( (y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1 ))

63/63 [==============================] - 0s 984us/step
[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [32]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1540   45]
 [ 225  190]]


0.865